In [2]:
import json
import numpy as np
import matplotlib.pyplot as plt
import zlib, base64
from skimage.io import imread
import cv2 as cv
import os
import re
import torch
import torchvision.transforms.functional as F
from PIL import Image
from torchvision import transforms
from torchvision.utils import draw_segmentation_masks
import skimage
from skimage import io

import itertools

Somehow some of the files ended up with a .jpg.jpg extension, which is a problem when your approach relies on matching images with targets using an exact filename match...

In [3]:
split_cracks_path = '/projects/wg-psel-ml/EL_images/eccoope/split_cracks'

mask_folder = os.path.join(split_cracks_path, 'ann_json')

In [5]:
img_folder = os.path.join(split_cracks_path, 'img_cropped', 'original')
for c in [c for c in os.listdir(img_folder) if 'jpg.jpg' in c]:
    name = re.search(r'(.+)\.jpg\.jpg', c).group(1)
    os.rename(os.path.join(img_folder, c), 
              os.path.join(img_folder, name + '.jpg'))

In [10]:
img_folder = os.path.join(split_cracks_path, 'img_cropped', 'all')
for c in [c for c in os.listdir(img_folder) if 'jpg.jpg' in c]:
    name = re.search(r'(.+)\.jpg\.jpg', c).group(1)
    os.rename(os.path.join(img_folder, c), 
              os.path.join(img_folder, name + '.jpg'))

In [11]:
img_folder = os.path.join(split_cracks_path, 'img_cropped', 'mirrored_x')
for c in [c for c in os.listdir(img_folder) if 'jpg.jpg' in c]:
    name = re.search(r'(.+)\.jpg\.jpg', c).group(1)
    os.rename(os.path.join(img_folder, c), 
              os.path.join(img_folder, name + '.jpg'))

In [12]:
img_folder = os.path.join(split_cracks_path, 'img_cropped', 'mirrored_xy')
for c in [c for c in os.listdir(img_folder) if 'jpg.jpg' in c]:
    name = re.search(r'(.+)\.jpg\.jpg', c).group(1)
    os.rename(os.path.join(img_folder, c), 
              os.path.join(img_folder, name + '.jpg'))

In [13]:
img_folder = os.path.join(split_cracks_path, 'img_cropped', 'mirrored_y')
for c in [c for c in os.listdir(img_folder) if 'jpg.jpg' in c]:
    name = re.search(r'(.+)\.jpg\.jpg', c).group(1)
    os.rename(os.path.join(img_folder, c), 
              os.path.join(img_folder, name + '.jpg'))

In [9]:
mask_folder = os.path.join(split_cracks_path, 'ann_json')
for c in [c for c in os.listdir(mask_folder) if 'jpg.jpg' in c]:
    name = re.search(r'(.+)\.jpg\.jpg\.json', c).group(1)
    os.rename(os.path.join(mask_folder, c), 
              os.path.join(mask_folder, name + '.jpg.json'))

In [20]:
for mask in os.listdir(mask_folder):
    matched = re.search(r'ASU3_2024-03-06 13_09_01.426_(.+\.jpg\.json)', mask)
    if matched is None:
        continue
    else:
        name = matched.group(1)
        os.rename(os.path.join(mask_folder, mask),
                  os.path.join(mask_folder, name))

In [21]:
os.listdir(mask_folder)

['2_EL_18.09.2023-15-31-57_unknownID_ASU Minisample_SHJ2_94.jpg.json',
 '1234 - Old TC for PID_Rear_4_Isc (EL Image).jpg.json',
 '1237 - NICE_SHJ_1_Isc (EL Image).jpg.json',
 '2_EL_18.09.2023-15-37-22_unknownID_ASU Minisample_SHJ4_100.jpg.json',
 '3_EL_18.09.2023-14-45-47_unknownID_ASU Minisample_09_51.jpg.json',
 '1_EL_18.09.2023-14-55-55_unknownID_ASU Minisample_06_62.jpg.json',
 '2_EL_2x2_NQ-01_20230915_125923.jpg.json',
 '3_EL_2x2_NQ-04_20230915_130134.jpg.json',
 '521 - TOPCon_4_rear_post (EL Image).jpg.json',
 '0_EL_18.09.2023-15-34-42_unknownID_ASU Minisample_SHJ3_97.jpg.json',
 '-1_EL_18.09.2023-15-53-07_unknownID_ASU Minisample_SHJ8_117.jpg.json',
 '2_EL_18.09.2023-14-51-50_unknownID_ASU Minisample_05_57.jpg.json',
 '3_EL_18.09.2023-15-40-40_unknownID_ASU Minisample_SHJ5_103.jpg.json',
 '1073 - GB Full Cell 2 No Indent initial (EL Image).jpg.json',
 '1176 - Ea_TC_EVA_1_Isc (EL Image).jpg.json',
 '1188 - Ea_TC_EVA_Isc (EL Image).jpg.json',
 '2_EL_18.09.2023-14-55-55_unknownID_A